In [4]:
import Trade
from ConnectDB import *
import datetime
import pytz
import pandas as pd

In [168]:
# example
AAL_long_order = Trade.go_long('AAL',100,100,120,80)

Paper API connected
Go long order submitted:
Order ID: 173b29dc-0871-4db5-b5fa-149505481c74
Entry point: 100
Take profit point: 120
Stop loss point: 79.2


In [169]:
# cancel all orders
Trade.api_access('paper').cancel_all_orders()

Paper API connected


In [4]:
# Check database connection,if not connected, connect it.
try:
    con
except NameError:
    # Connect to database
    con,cur = connect_data_base()
    print('Initial connection to PostgreSQL DataBase')
else:
    if con.closed !=0:
    # Connect to database
        con,cur = connect_data_base()
        print('Reconnected to PostgreSQL DataBase')
    print(con.closed)
        

0


In [7]:
# Connect to database
con,cur = connect_data_base()

DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit


In [8]:
# Fuction for acquiring data from database.
def get_data(data_type = 'stock_bar'):
    # acquire wall street date
    def get_wall_street_date():
        """
        return wall street date
        """
        tz = pytz.timezone('America/New_York')
        return datetime.datetime.now(tz).strftime('%Y-%m-%d')

    # acquire latest data from realtime database (last 60 minites)
    wall_street_date = get_wall_street_date()
    table_name = "_" + wall_street_date.replace('-','_') + f'_{data_type}'
    cur.execute("SELECT * FROM {} ORDER BY time DESC LIMIT 60".format(table_name))
    data = cur.fetchall()

    # Acquire table columns names
    cur.execute("SELECT * FROM information_schema.columns WHERE table_schema = 'public' AND table_name   = '{}'".format(table_name))
    rows = cur.fetchall()
    table_columns = []
    for row in rows:
        table_columns.append(row[3])


    data_df = pd.DataFrame(data,columns= table_columns)
    return data_df

In [9]:
# call the get_data function and show the database.
data_df = get_data(data_type = 'crypto_bar')
data_df.head()

,symbol,time,type,close,high,low,open,trade,volume,vwap
0,BTC/USD,2022-11-13 00:03:00-05:00,b,16831.13,16834.18,16825.92,16834.18,26,0.329199,16830.6865059128
1,BTC/USD,2022-11-13 00:02:00-05:00,b,16830.48,16833.23,16825.12,16827.49,48,1.057035,16826.9901836741
2,BTC/USD,2022-11-13 00:01:00-05:00,b,16826.72,16833.8,16819.71,16827.08,166,3.869651,16827.9613775118
3,BTC/USD,2022-11-13 00:00:00-05:00,b,16832.62,16834.53,16827.08,16833.36,25,2.22605,16829.6153131376
